In [18]:
import glob
import pandas as pd
import os
import numpy as np
from IPython.display import HTML

In [9]:
!ls /home/pierre/scVI/results/models_comparison/

brain_small  mixed_25_dataset  mixed_75_dataset  retina
cortex	     mixed_50_dataset  nb_dataset	 zinb_dataset


In [20]:
data_files = glob.glob('/home/pierre/scVI/results/models_comparison/{}/*.csv'.format('nb_dataset'))
print(data_files)
data_folder = '/home/pierre/scVI/results/models_comparison/{}'.format('nb_dataset')
data_names = ['zifa_full', 'nb', 'zinb']

['/home/pierre/scVI/results/models_comparison/nb_dataset/zifa_half_nb_dataset.csv', '/home/pierre/scVI/results/models_comparison/nb_dataset/nb_nb_dataset.csv', '/home/pierre/scVI/results/models_comparison/nb_dataset/zinb_nb_dataset.csv']


In [11]:
dfs = []
for data_name, f in zip(data_names, data_files):
    my_df = pd.read_csv(f, sep='\t')
    my_df.loc[:, 'model'] = data_name
#     my_df.columns = [data_name + '_' col_name for col_name in my_df.columns]
    dfs.append(my_df)
df = pd.concat(dfs, axis=0)
# df.info()

In [12]:
metrics = ['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat', 't_ratio_ks_stat', 't_cv_ks_stat']
h1_hypothesis = ['larger', 'larger', 'larger', 'larger', 'larger']
df_nb = df.loc[df.model=='nb', metrics]

from statsmodels.stats.weightstats import ttest_ind

def get_pvals(gby):
    my_df = gby[metrics]
    assert my_df.shape[1] == len(metrics)
    pvals = []
    for idx, alternative in enumerate(h1_hypothesis):
        assert len(df_nb.values[:, idx]) != len(h1_hypothesis)
        _, pval, _ = ttest_ind(df_nb.values[:, idx], my_df.values[:, idx], alternative=alternative)
        pvals.append(pval)
    return np.array(pvals)

pvals = df.groupby('model').apply(get_pvals)
pvals = (pvals
         .apply(lambda x: pd.Series(x))
         .T)
pvals = pvals.rename(index={idx: met for (idx,met) in enumerate(metrics)})

/home/pierre/miniconda3/envs/scvi/lib/python3.6/site-packages/statsmodels/stats/weightstats.py:616: RuntimeWarning: invalid value encountered in double_scalars
  tstat = (value1 - value2 - diff) / std_diff


In [13]:
def get_summary(gby):
    res = {}
    res['mean'] = gby.mean()
    res['std'] = gby.std()
    res['pvals'] = pvals[gby.name]
    return pd.DataFrame(res).T

# df_summary = df.groupby('model')['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat',
#                                  't_ratio_ks_stat', 't_cv_ks_stat'].agg(['mean', 'std']).T
df_summary = df.groupby('model')[metrics].apply(get_summary)
df_summary


def my_styler(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    print(val)
    style = 'background-color: yellow' if val.pvals < 0.05 else ''
    return [style]

df_summary = df_summary.stack().unstack(1).sort_index(level=1).swaplevel()
df_summary.style.apply(my_styler, subset=['pvals'], axis=1)
# df_summary.loc[pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :]].style.background_gradient()
# df_summary.style.apply(my_styler, subset=pd.IndexSlice[:, 'pvals'], axis=0)
# df_summary = df_summary.style.apply(my_styler, subset=pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :], axis=1).render()
# df_summary.pivot(index=['model', ['mean', 'std', 'pvals']], columns=metrics)

pvals    0.5
Name: (imputation_median_imputation_score, nb), dtype: float64
pvals    0.980205
Name: (imputation_median_imputation_score, zifa_half), dtype: float64
pvals    0.970934
Name: (imputation_median_imputation_score, zinb), dtype: float64
pvals    0.5
Name: (ll_ll, nb), dtype: float64
pvals    5.129570e-10
Name: (ll_ll, zifa_half), dtype: float64
pvals    0.138175
Name: (ll_ll, zinb), dtype: float64
pvals   NaN
Name: (t_cv_ks_stat, nb), dtype: float64
pvals    0.077221
Name: (t_cv_ks_stat, zifa_half), dtype: float64
pvals   NaN
Name: (t_cv_ks_stat, zinb), dtype: float64
pvals    0.5
Name: (t_dropout_ks_stat, nb), dtype: float64
pvals    1.0
Name: (t_dropout_ks_stat, zifa_half), dtype: float64
pvals    1.0
Name: (t_dropout_ks_stat, zinb), dtype: float64
pvals    0.5
Name: (t_ratio_ks_stat, nb), dtype: float64
pvals    1.0
Name: (t_ratio_ks_stat, zifa_half), dtype: float64
pvals    0.999572
Name: (t_ratio_ks_stat, zinb), dtype: float64


In [14]:
# df_summary.loc[pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :]].style.background_gradient()

In [21]:
text_file = open(os.path.join(data_folder, 'latex_table.txt'), "w")
text_file.write(df_summary.to_latex())
text_file.close()